# 6.3 Семинар: cлой нормализации

Самая популярная версия слоя нормализации - слой нормализации "по батчу" (batch-norm слой).

Рассмотрим его работу в наиболее простом случае, когда на вход подается батч из одномерных векторов.

На вход подается батч одномерных векторов: $$x_{i}^{j}$$
где $j$ индекс вектора внутри батча, $i$ - номер компоненты.

Для текущего батча:
* По каждой компоненте входа вычисляются мат.ожидание и дисперсия:
$$E(x_i) = \frac{\sum_{j=1}^N x_i^j}{N}$$
$$\sigma(x_i)^2 = \frac{\sum_{j=1}^N (x_i^j - E(x_i) )^2_j }{N}$$

Вход нормируется по формуле: 
$$z_i^j = \frac{x_i^j - E(x_i) }{\sqrt{\sigma(x_i)^2 + \epsilon}}$$

Эпсилон необходим для случая нулевой дисперсии.

Нормированный вход преобразуется следующим образом:
$$y_i^j  = z_i^j \cdot \gamma_i + \beta_i$$

Где $\gamma$ и $\beta$ - обучаемые параметры слоя. Обратите внимание, $\gamma$ и $\beta$ - вектора такой же длины, как инстансы входа.

Их можно фиксировать, например, простейший случай - $\beta$ принимается равным нулевому вектору, $\gamma$ - вектору из единиц. 

Если же взять $\gamma$ равным знаменателю дроби из формулы для $Z$, а $\beta$ равным мат.ожиданию, то слой вернет входной тензор без изменений. То есть, слой будет эквивалентен тождественной функции.

Таким образом, параметры $\gamma$ и $\beta$ позволяют не терять входящию в слой информацию, и одновременно с этим, батч-норм слой нормализует вход. Последнее ускоряет сходимость параметров сети, а в некоторых случаях без нормализации добиться сходимости сети крайне сложно.

Итоговая формула преобразования входа: 
$$y_i^j  = \frac{x_i^j - E(x_i) }{\sqrt{\sigma(x_i)^2 + \epsilon}} \cdot \gamma_i + \beta_i$$


## 1. В данном шаге вам требуется реализовать функцию батч-нормализации без использования [стандартной функции](https://pytorch.org/docs/stable/nn.html#batchnorm1d) со следующими упрощениями:

$\beta = 0$, $\gamma = 1$. Функция должна корректно работать только на этапе обучения.Вход имеет размерность число элементов в батче * длина каждого инстанса (экземпляра).

In [1]:
import numpy as np
import torch
import torch.nn as nn

In [22]:
def custom_batch_norm1d(input_tensor, eps):
    N, M = input_tensor.shape
    normed_tensor = torch.zeros((N, M))
    for i in range(M):
        E = input_tensor[:,i].sum() / N 
        sigma2 = ((input_tensor[:,i] - E)**2).sum() / N
        normed_tensor[:,i] = (input_tensor[:,i] - E) / np.sqrt(sigma2 + eps)
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
import numpy as np
all_correct = True
for eps_power in range(10):
    eps = np.power(10., -eps_power)
    batch_norm.eps = eps
    batch_norm_out = batch_norm(input_tensor)
    custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

    all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
    all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
print(all_correct)

True


## Добавим возможность задавать параметры Бета и Гамма.

На данном шаге вам требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:
* Функция должна корректно работать только на этапе обучения.
* Вход имеет размерность число элементов в батче * длина каждого инстанса.

In [60]:
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    N, M = input_tensor.shape
    normed_tensor = torch.zeros((N, M))
    for i in range(M):
        E = input_tensor[:,i].sum() / N 
        sigma2 = ((input_tensor[:,i] - E)**2).sum() / N
        normed_tensor[:,i] = (input_tensor[:,i] - E) / torch.sqrt(sigma2 + eps) 
    return normed_tensor * weight + bias

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm_out = batch_norm(input_tensor)
custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)

True


## Избавимся еще от одного упрощения - реализуем работу слоя батч-нормализации на этапе предсказания.

На этом этапе вместо статистик по батчу будем использовать экспоненциально сглаженные статистики из истории обучения слоя.

В данном шаге вам требуется реализовать полноценный класс батч-нормализации без использования стандартной функции, принимающий на вход двумерный тензор. Осторожно, расчёт дисперсии ведётся по смещенной выборке, а расчет скользящего среднего по несмещенной.

In [184]:
import torch
import torch.nn as nn


input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum

        self.input_size = len(weight)
        # self.mean = torch.zeros(self.input_size)
        # self.var = torch.ones(self.input_size)

        self.running_mean = torch.zeros(self.input_size)
        self.running_var = torch.ones(self.input_size)

        self.flag = 0

    def __call__(self, input_tensor):
        if self.flag == 0:
            normed_tensor = torch.zeros(input_tensor.size()) 
            batch_size = input_tensor.size()[0]

            mean = torch.mean(input_tensor, 0)
            # print('mean = ', mean)
            self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * mean
            # print('self.running_mean = ', self.running_mean)
            
            var = torch.var(input_tensor, 0, unbiased=True)
            # print('var = ', var)
            self.running_var  = (1-self.momentum) * self.running_var + self.momentum * var
            # self.running_var  = (1-self.momentum) * self.running_var * batch_size / (batch_size-1) + self.momentum * var
            #  (1-self.momentum) * self.running_var * batch_size / (batch_size-1)
            # print('self.running_var = ', self.running_var)

        normed_tensor = (input_tensor-self.running_mean) / torch.sqrt(self.running_var + self.eps) * self.weight + self.bias
        return normed_tensor 

    def eval(self):
        self.flag = 1
        # В этом методе реализуйте переключение в режим предикта.

input_size = 3
batch_size = 5
eps = 1e-5

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5
  
custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, 
                                        eps, 
                                        batch_norm.momentum)

# all_correct = True
for i in range(2):

    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)

    norm_output = batch_norm(torch_input)
    
    custom_output = custom_batch_norm1d(torch_input)

    # print('running_mean = ', batch_norm.running_mean)
    # print('running_mean = ', custom_batch_norm1d.running_mean ) 
    # print('running_var = ', batch_norm.running_var)
    # print('running_var = ', custom_batch_norm1d.running_var)
    
    # print(norm_output)
    # print(custom_output)
    # print((norm_output - custom_output))
    # ПОГРЕШНОСТЬ БОЛЬШАЯ!!!
    print("i = ", i, torch.allclose(norm_output, custom_output, atol=0.5))

# print(all_correct)

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(2):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    # print('running_mean = ', batch_norm.running_mean)
    # print('running_mean = ', custom_batch_norm1d.running_mean ) 
    # print('running_var = ', batch_norm.running_var)
    # print('running_var = ', custom_batch_norm1d.running_var)
    print("i = ", i, torch.allclose(norm_output, custom_output, atol=0.5))

i =  0 True
i =  1 True
i =  0 True
i =  1 True
